In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
np.random.seed(0)


def mask(T=200):
    mask = np.zeros(T)
    indices = np.random.permutation(np.arange(T))[:2]
    mask[indices] = 1
    return mask


def toy_problem(N=10, T=200):
    signals = np.random.uniform(low=0.0, high=1.0, size=(N, T))
    masks = np.zeros((N, T))
    for i in range(N):
        masks[i] = mask(T)

    data = np.zeros((N, T, 2))
    data[:, :, 0] = signals[:]
    data[:, :, 1] = masks[:]
    target = (signals * masks).sum(axis=1).reshape(N, 1)

    return (data, target)

In [3]:
'''
데이터를 생성한다
'''
N = 10000
T = 200
maxlen = T

X, Y = toy_problem(N=N, T=T)

N_train = int(N * 0.9)
N_validation = N - N_train


In [4]:


'''
모델을 설정한다
'''
n_in = len(X[0][0])  # 2
n_hidden = 100
n_out = len(Y[0])  # 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)


early_stopping = EarlyStopping(monitor='loss', patience=100, verbose=1)

model = Sequential()
model.add(LSTM(n_hidden,
               kernel_initializer=weight_variable,
               input_shape=(maxlen, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error',
              optimizer=optimizer)


In [ ]:

'''
모델을 학습시킨다
'''
epochs = 1000
batch_size = 100

hist = model.fit(X, Y,
                 batch_size=batch_size,
                 epochs=epochs,
                 callbacks=[early_stopping])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.2944
Epoch 2/1000
10000/10000 [==============================] - 18s 2ms/step - loss: 0.1682
Epoch 3/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.1684
Epoch 4/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.1679
Epoch 5/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.1680: 0s - loss: 0
Epoch 6/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.1680
Epoch 7/1000
10000/10000 [==============================] - 19s 2ms/step - loss: 0.1681
Epoch 8/1000
10000/10000 [==============================] - 20s 2ms/step - loss: 0.1687
Epoch 9/1000
10000/10000 [==============================] - 20s 2ms/step - loss: 0.1683
Epoch 10/1000
10000/10000 [==============================] - 20s 2ms/step - loss: 0.1690
Epoch 11

10000/10000 [==============================] - 62s 6ms/step - loss: 0.1680
Epoch 83/1000
10000/10000 [==============================] - 63s 6ms/step - loss: 0.1682
Epoch 84/1000
10000/10000 [==============================] - 64s 6ms/step - loss: 0.1681
Epoch 85/1000
10000/10000 [==============================] - 65s 7ms/step - loss: 0.1683
Epoch 86/1000
10000/10000 [==============================] - 65s 6ms/step - loss: 0.1682
Epoch 87/1000
10000/10000 [==============================] - 75s 8ms/step - loss: 0.1682
Epoch 88/1000
10000/10000 [==============================] - 72s 7ms/step - loss: 0.1678
Epoch 89/1000
10000/10000 [==============================] - 70s 7ms/step - loss: 0.1680
Epoch 90/1000
10000/10000 [==============================] - 75s 8ms/step - loss: 0.1679
Epoch 91/1000
10000/10000 [==============================] - 74s 7ms/step - loss: 0.1681
Epoch 92/1000
10000/10000 [==============================] - 76s 8ms/step - loss: 0.1684
Epoch 93/1000
10000/10000 [========

In [ ]:

'''
학습이 진행되는 상황을 가시화한다
'''
loss = hist.history['loss']

plt.rc('font', family='serif')
fig = plt.figure()
plt.plot(range(len(loss)), loss, label='loss', color='black')
plt.xlabel('epochs')
plt.show()
plt.savefig(__file__ + '.eps')